In [1]:
import os
import zipfile
import time
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import random
from sklearn.model_selection import StratifiedKFold,train_test_split
from tqdm import tqdm

import torch.optim as optim
import torch.utils.data as data
from torchvision import models, transforms
from torchvision.utils import make_grid
from PIL import Image
from torch.utils.data import Dataset, random_split, DataLoader


%matplotlib inline

import glob
import re

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import PIL
from PIL import Image

In [2]:
# Config
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCH=5
model = 'VGG-16' #'VGG-19',ResNet-18 ResNet-50

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [4]:
def natural_key(string_):
    """
    Define sort key that is integer-aware
    """
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

#These are the directory on kaggle. I firstly unzipped the data and generated a new folder to save the data.
#https://www.kaggle.com/competitions/dogs-vs-cats-redux-kernels-edition/data
TRAIN_DIR = '../input/data-cat-and-dogs/train/'
TEST_DIR = '../input/data-cat-and-dogs/test/'

train_cats = sorted(glob.glob(os.path.join(TRAIN_DIR, 'cat*.jpg')), key=natural_key)
train_dogs = sorted(glob.glob(os.path.join(TRAIN_DIR, 'dog*.jpg')), key=natural_key)
train_all = train_cats + train_dogs

test_all = sorted(glob.glob(os.path.join(TEST_DIR, '*.jpg')), key=natural_key)
y_all = [train_all[i].split('/')[-1].split('.')[0] for i in range(len(train_all))]

In [5]:
# Data Augumentation
class ImageTransform():
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            #Compose:
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                #Randomly resize + rescale
                transforms.RandomHorizontalFlip(),
                #horizontal flip
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
                #standerlize
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [6]:
# Dataset
class DogvsCatDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train'):    
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        
        img_transformed = self.transform(img, self.phase)
        
        # Get Label
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0

        return img_transformed, label

In [7]:
class Custom_model(nn.Module):
    def __init__(self,net): #需要输入预训练模型最后一层的维度
        super(Custom_model,self).__init__()
        self.pretrained_model = net
        self.hidden_state = 1000
        self.linear1 = nn.Linear(self.hidden_state,512)
        self.dropout = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(512,1)
    
    def forward(self,x):
        x = F.relu(self.pretrained_model(x))
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        x = torch.sigmoid(x).squeeze(-1)
        #print(x)
        return x

In [8]:
def train_model(net, dataloader_dict, criterion, optimizer, num_epoch):
    
    since = time.time()
    best_model_wts = copy.deepcopy(net.state_dict())
    best_acc = 0.0
    net = net.to(device)
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'val']:
            
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    preds = torch.tensor([round(float(i)) for i in outputs]).to(device)
                    #_, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.type_as(outputs))

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                print('A better model in current epoch! Saved')
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(net.state_dict())
                torch.save({'model': net.state_dict(),
                            'predictions': preds},
                            f"model_best.pth")
            
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    net.load_state_dict(best_model_wts)
    return net

In [12]:
train,val,train_y,val_y = train_test_split(train_all,y_all,test_size=0.1,stratify=y_all)

################if you change the config, you should also change the model used here.
res = models.vgg16(pretrained=True) 


for param in res.parameters():
    param.requires_grad=False
# build the model
model_final = Custom_model(net=res)
# Put the model to GPU
model_final= model_final.to(device)
# cost_function
cost_function = nn.BCELoss()  
# optimizer
optimizer_ft = optim.Adam([param for param in model_final.parameters() if param.requires_grad],lr=0.009)
# learning rate scheduler

# Dataset
train_dataset = DogvsCatDataset(train, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = DogvsCatDataset(val, transform=ImageTransform(size, mean, std), phase='val')

# DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloader_dict = {'train': train_dataloader, 'val': val_dataloader}

net = train_model(model_final, dataloader_dict, cost_function, optimizer_ft, EPOCH)



Epoch 1/10
--------------------


100%|██████████| 79/79 [00:18<00:00,  4.32it/s]s]


val Loss: 0.0673 Acc: 0.9740
A better model in current epoch! Saved
Epoch 2/10
--------------------


100%|██████████| 704/704 [02:19<00:00,  5.03it/s]


train Loss: 0.1294 Acc: 0.9515


...

val Loss: 0.0644 Acc: 0.9712
Epoch 10/10
--------------------


100%|██████████| 704/704 [02:19<00:00,  5.04it/s]


train Loss: 0.1093 Acc: 0.9580


100%|██████████| 79/79 [00:16<00:00,  4.86it/s]

val Loss: 0.0688 Acc: 0.9728
Training complete in 26m 15s
Best val Acc: 0.978800


In [ ]:
def predict_on_loader(test,model,device):
    print('Start predicting.....')
    model.eval()
    # Prediction
    id_list = []
    pred_list = []
    
    with torch.no_grad():
        for test_path in tqdm(test):
            #print(test_path)
            img = Image.open(test_path)
            _id = int(test_path.split('/')[-1].split('.')[0])
            transform = ImageTransform(size, mean, std)
            img = transform(img, phase='val')
            img = img.unsqueeze(0)
            img = img.to(device)

            outputs = model(img)
            preds = float(outputs)
            id_list.append(_id)
            pred_list.append(preds)
    
    return id_list,pred_list

In [ ]:
all_predict = []

################if you change the config, you should also change the model used here.
model = Custom_model(net = models.vgg16(pretrained=True))

state = torch.load(f"model_best.pth")
model.load_state_dict(state['model'])
id_list,predictions = predict_on_loader(test_all,model_final,device)
all_predict.append(predictions)
pred_list = pd.DataFrame(all_predict).mean().to_list()

In [ ]:
res = pd.DataFrame({
    'id': id_list,
    'label': pred_list
})

res.sort_values(by='id', inplace=True)
res.reset_index(drop=True, inplace=True)

res.to_csv('submission.csv', index=False)